In [1]:
import os

import gdal
from plio.io.io_gdal import GeoDataset

/usgs/apps/anaconda/envs/autocnet/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# setup the paths to cubes and tiffs
hirise_cub1 = "ESP_023524_1985_1m_o_isis3.cub"
hirise_cub2 = "ESP_048908_1985_1m_o_isis3.cub"
ctx_cub1 = "F05_037607_2008_XN_20N282W_v6_PosAndVelAndAngles_20m_o.cub"
ctx_cub2 = "J03_045994_1986_XN_18N282W_v6_20m_o.cub"
ctx_cub3 = "F05_037607_2008_XN_20N282W_v6_PosAndVelAndAngles_6m_o.cub"
ctx_cub4 = "J03_045994_1986_XN_18N282W_v6_6m_o.cub"
hrsc_cub = "H5270_0000_ND4.IMG"

In [30]:
def translate_image(in_image, out_image, overlap_points, res, in_srs, out_srs):
    # if not os.path.exists(tiff_image_mod1):
    ul, lr = overlap_points

    minx, miny= ul
    maxx, maxy = lr

    fp = gdal.Translate(out_image, in_image, srcWin = [minx, miny, maxx - minx, maxy - miny])
    del(fp)

#     fp = gdal.Warp(out_image, out_image, targetAlignedPixels=True, xRes = res, yRes = res, resampleAlg=resample, srcSRS=src_in_srs, dstSRS=out_srs)
    fp = gdal.Warp(out_image, out_image, targetAlignedPixels=True, xRes = res, yRes = res, resampleAlg=resample)
    del(fp)

In [31]:
hirise_basepath = '/home/acpaquette/repos/mars2020_trn/TestData/HiRISE_Jezero/'
ctx_20_basepath = '/home/acpaquette/repos/mars2020_trn/TestData/CTX_Jezero/20m_ctx/'
ctx_6_basepath = '/home/acpaquette/repos/mars2020_trn/TestData/CTX_Jezero/6m_ctx/'
hrsc_basepath = '/home/acpaquette/repos/mars2020_trn/TestData/HRSC_Jezero/'

cub_image1 = os.path.join(hirise_basepath, hirise_cub1)
cub_image2 = os.path.join(ctx_20_basepath, ctx_cub1)

# cub_image1 = os.path.join(hrsc_basepath, hrsc_cub)
# cub_image2 = os.path.join(ctx_20_basepath, ctx_cub1)

# cub_image1 = os.path.join(ctx_20_basepath, ctx_cub1)
# cub_image2 = os.path.join(ctx_20_basepath, ctx_cub2)

tiff_image1 = os.path.splitext(cub_image1)[0] + '.tiff'
tiff_image2 = os.path.splitext(cub_image2)[0] + '.tiff'

src_in_srs = gdal.Info(cub_image1, format='json')['coordinateSystem']['wkt']
dst_in_srs = gdal.Info(cub_image2, format='json')['coordinateSystem']['wkt']

if 'Equirectangular' in src_in_srs:
    src_out_srs = src_in_srs
else:
    if 'Equirectangular' in dst_in_srs:
        src_out_srs = dst_in_srs

if 'Equirectangular' in dst_in_srs:
    dst_out_srs = dst_in_srs
else:
    if 'Equirectangular' in src_in_srs:
        dst_out_srs = src_in_srs
        
# print(src_in_srs)
# print(dst_in_srs)

resample = 'bilinear'
res = 20

# Get all geo data for the two co-registered tiffs
cub_geo1 = GeoDataset(cub_image1)
cub_geo2 = GeoDataset(cub_image2)

overlap = cub_geo1.compute_overlap(cub_geo2)[0]

overlap_points1 = [cub_geo1.latlon_to_pixel(i[1], i[0]) for i in overlap][::2]
overlap_points2 = [cub_geo2.latlon_to_pixel(i[1], i[0]) for i in overlap][::2]

In [32]:
translate_image(cub_image1, tiff_image1, overlap_points1, res, src_in_srs, src_out_srs)

In [33]:
translate_image(cub_image2, tiff_image2, overlap_points2, res, dst_in_srs, dst_out_srs)

In [ ]:
# src_in_srs = gdal.Info(tiff_image1, format='json')['coordinateSystem']['wkt']
# dst_in_srs = gdal.Info(tiff_image2, format='json')['coordinateSystem']['wkt']
# print(src_in_srs)
# print(dst_in_srs)